<div class="alert alert-info" role="alert">
<h1 class = "alert-heading"> Proyecto: *Diagnosing Sickle Cell Anemia*</h1>
<hr>

**Biopython**
<p>*Paulina Rosales Becerra*</p>
<p>LCG - UNAM</p>
03 de febrero, 2021


</div>

<a id='indice'></a>
## Índice
[Objetivo](#0)
1. [*Bio.Entrez*: Obtención de secuencias de ADN y proteicas con ](#1)
    1. [Búsqueda de identificadores en *NCBI*](#1.1)
    1. [Recuperar la secuencia utilizando identificadores](#1.2)
    1. [Obtener una única entrada *GenBank*](#1.3)
    1. [Escribir en un archivo *output*](#1.4)
    1. [Recuperar y escribir múltiples entradas *GenBank*](#1.5)
2. [*Bio.SeqIO*: Procesamiento de secuencias](#2)
    1. [Leer archivo *GenBank*](#2.1)
    1. [Imprimir la información de una secuencia](#2.2)
    1. [Escribir un archivo FASTA](#2.3)
    1. [Imprimir la información de múltiples secuencias](#2.4)
    1. [Filtrar las entradas de secuencia](#2.5)
3. [*Bio.Seq* y *Bio.SeqRecord*: Extracción de exones](#3)
    1. [Secuencia ADN](#3.1)
    1. [Transcribir ADN a ARN](#3.2)
    1. [Traducir ARN a proteína](#3.3)
    1. [Analizar las anotaciones de *beta-globina*](#3.4)
    1. [Extraer *features* de la secuencia](#3.5)
    1. [Extraer exones](#3.6)
    1. [Extraer la secuencia proteica de *beta-globina*](#3.7)
    1. [Resultados](#3.8)
4. [Coincidencia de patrones: Identificar sitios de restricción](#4)
    1. [Familiarización con expresiones regulares](#4.1)
    1. [Buscar el codón de inicio](#4.2)
    1. [Buscar sitio de restricción](#4.3)
    1. [Utilizar más sitios de restricción](#4.4)
    1. [Reemplazar los caracteres faltantes](#4.5)

[Resultados: Archivos *output* generados](#5)

[Conclusiones](#6)

<a id='0'></a>
## Objetivo
[Regresar al índice](#indice)

El objetivo de este ejercicio propuesto por [Kristian Rotcher (2015)](https://krother.gitbooks.io/biopython-tutorial/content/sicklecell.html) es desarrollar una prueba experimental que revele si un paciente padece anemia falciforme *(sickle cell anemia)* hereditaria. La prueba diagnóstico deberá utilizar una enzima de restricción en muestras de ADN de pacientes. Para el trabajo de la prueba necesitaremos saber cuál es la diferencia exacta para probar en contra. 

La idea es comparar ADN y secuencias proteicas de globina *sickle cell* y globina sana, e utilizar diferentes enzimas de restricción en ellas.

Recordemos que el desarrollo de este proyeecto será a partir de herramientaas de *Biopython*

<a id='1'></a>
## 1. *Bio.Entrez*
### Obtención de secuencias de ADN y proteicas con 
[Regresar al índice](#indice)

Obtención de cADN de *sickle cell Hb* + genes de *no-sickle Hb* con <code>Bio.Entrez</code>

<a id='1.1'></a>
### 1.1 Búsqueda de identificadores en *NCBI*
[Regresar al índice](#indice)
 
 Extracción de identificadores obtenidos por <code>Entrez.esearch</code>

In [14]:
from Bio import Entrez

Entrez.email = "prosales@lcg.unam.mx" 
search = 'sickle AND homo sapiens AND globin NOT chromosome'
# Búsqueda
search_handle = Entrez.esearch(db = "nucleotide", term = search, usehistory = 'y')
search_results = Entrez.read(search_handle)
search_handle.close()
identifiers = search_results['IdList']

print('Se obtuvieron {} identificadores de la búsqueda de {} en NCBI:\n'.format(len(identifiers), search))
for result in identifiers:
    print(result)

Se obtuvieron 7 identificadores de la búsqueda de sickle AND homo sapiens AND globin NOT chromosome en NCBI:

1868032479
1515564438
179408
224959855
2168937
183859
183844


<a id='1.2'></a>
### 1.2 Recuperar la secuencia utilizando identificadores
[Regresar al índice](#indice)

Utilizar los identificadores del 1.1 para extraer la secuencia con <code>Entrez.efetch</code>

In [18]:
# Extracción en xml
fetch_handle = Entrez.efetch(db="nucleotide", rettype="fasta", id =identifiers, retmode="xml")
fetch_results = Entrez.read(fetch_handle)
fetch_handle.close()

print('Se obtuvieron {} secuencias a partir de los identificadores\n'.format(len(fetch_results)))
for result in fetch_results:
    print('ID:\t{}\nDef:\t{}\n'.format(result['TSeq_accver'], result['TSeq_defline']))


Se obtuvieron 7 secuencias a partir de los identificadores

ID:	MN609913.1
Def:	Homo sapiens A-gamma globin (HBG1) gene, exon 1 and partial cds

ID:	MH580289.1
Def:	Homo sapiens voucher ATGLAB 2018103 hemoglobin beta subunit (HBB) gene, exon 3 and partial cds

ID:	M25079.1
Def:	Human sickle cell beta-globin mRNA, complete cds

ID:	FJ766333.1
Def:	Homo sapiens A-gamma globin (HBG1) gene, promoter region

ID:	E00658.1
Def:	Part of DNA encoding beta-globin gene

ID:	M33706.1
Def:	Human hemoglobin DNA with a deletion causing Indian delta-beta thalassemia

ID:	M37467.1
Def:	Human hemoglobin-related sequence across the breakpoint for Indian delta-beta thalassemia



<a id='1.3'></a>
### 1.3 Obtener una única entrada *GenBank*
[Regresar al índice](#indice)

Obtener el identificador de la globina *sickle cell*  de manera manual para descargar la entrada completa de *GenBank* sólo para la secuencia del <code>efecth</code>

In [19]:
# Estracción a partir de un ID particular
gb_handle = Entrez.efetch(db = "nucleotide", id = "M25079.1", rettype = "gb", retmode = "text")
gb_record = gb_handle.read()
gb_handle.close()
print(gb_record)

LOCUS       HUMBETGLA                468 bp    mRNA    linear   PRI 27-APR-1993
DEFINITION  Human sickle cell beta-globin mRNA, complete cds.
ACCESSION   M25079
VERSION     M25079.1
KEYWORDS    .
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.
REFERENCE   1  (bases 1 to 468)
  AUTHORS   Marotta,C.A., Forget,B.G., Cohen-Solal,M. and Weissman,S.M.
  TITLE     Nucleotide sequence analysis of coding and noncoding regions of
            human beta-globin mRNA
  JOURNAL   Prog. Nucleic Acid Res. Mol. Biol. 19, 165-175 (1976)
   PUBMED   1019344
COMMENT     Original source text: Human sickle cell, cDNA to mRNA.
FEATURES             Location/Qualifiers
     source          1..468
                     /organism="Homo sapiens"
                     /mol_type="mRNA"
                     /db_xref="ta

<a id='1.4'></a>
### 1.4 Escribir en un archivo *output*
[Regresar al índice](#indice)

Guardar la entrada de *GenBank* en el archivo <code>sickle.gb</code>

In [20]:
filename = 'outputs/sickle.gb'
with open(filename, "w") as file:
        file.write(gb_record)

In [21]:
%%bash
less outputs/sickle.gb

LOCUS       HUMBETGLA                468 bp    mRNA    linear   PRI 27-APR-1993
DEFINITION  Human sickle cell beta-globin mRNA, complete cds.
ACCESSION   M25079
VERSION     M25079.1
KEYWORDS    .
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.
REFERENCE   1  (bases 1 to 468)
  AUTHORS   Marotta,C.A., Forget,B.G., Cohen-Solal,M. and Weissman,S.M.
  TITLE     Nucleotide sequence analysis of coding and noncoding regions of
            human beta-globin mRNA
  JOURNAL   Prog. Nucleic Acid Res. Mol. Biol. 19, 165-175 (1976)
   PUBMED   1019344
COMMENT     Original source text: Human sickle cell, cDNA to mRNA.
FEATURES             Location/Qualifiers
     source          1..468
                     /organism="Homo sapiens"
                     /mol_type="mRNA"
                     /db_xref="ta

<a id='1.5'></a>
### 1.5 Recuperar y escribir múltiples entradas *GenBank*
[Regresar al índice](#indice)

Combina <code>esearch</code> y <code>efetch</code> para regresar entradas de la secuencias de genes de la familia globina en humanos. Encuentra las *keywords* apropiadas para limitar la búsqueda a *beta-globina* y secuencias codificantes completas. Escribe el resultado en un documento

In [22]:
search ='(beta-globin[All Fields]) AND (Homo sapiens[Organism]) AND (complete cds[All Fields])'
out_file = 'outputs/beta-globins.gb'
out_handle = open(out_file, "w")
batch_size = 3

# Búsqueda y extracción de múltiples secuencias
print('\nDownloading data from {}'.format(search))
search_handle = Entrez.esearch(db = "nucleotide", term = search, usehistory = 'y')
search_results = Entrez.read(search_handle)
search_handle.close()
count = int(search_results["Count"])
webenv = search_results["WebEnv"]
query_key = search_results["QueryKey"]

fetch_handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="text",
                                             webenv=webenv, query_key=query_key)

data = fetch_handle.read()
fetch_handle.close()
out_handle.write(data)
out_handle.close()

print('\nDownload finished, fetched {} sequences saved at {}'.format(count, out_file))



Download finished, fetched 132 sequences saved at outputs/beta-globins.gb


In [23]:
%%bash
head outputs/beta-globins.gb

LOCUS       MG657341                1606 bp    DNA     linear   PRI 22-OCT-2018
DEFINITION  Homo sapiens hemoglobin subunit beta (HBB) gene, complete cds.
ACCESSION   MG657341
VERSION     MG657341.1
KEYWORDS    .
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.


<a id='2'></a>
## 2. *Bio.SeqIO*: Procesamiento de secuencias 
[Regresar al índice](#indice)

Leer, escribir y filtrar archivos secuencia.

<a id='2.1'></a>
### 2.1 Leer archivo *GenBank*
[Regresar al índice](#indice)

Leer el archivo <code>sickle.gb</code> usando <code>SeqIO</code>

In [3]:
from Bio import SeqIO

for gb_record in SeqIO.parse('outputs/sickle.gb', 'genbank'):
    print(gb_record)

ID: M25079.1
Name: HUMBETGLA
Description: Human sickle cell beta-globin mRNA, complete cds
Number of features: 2
/date=27-APR-1993
/data_file_division=PRI
/keywords=['']
/source=Homo sapiens (human)
/molecule_type=mRNA
/accessions=['M25079']
/comment=Original source text: Human sickle cell, cDNA to mRNA.
/organism=Homo sapiens
/topology=linear
/sequence_version=1
/references=[Reference(title='Nucleotide sequence analysis of coding and noncoding regions of human beta-globin mRNA', ...)]
/taxonomy=['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo']
Seq('ATGGTNCAYYTNACNCCNGTGGAGAAGTCYGCYGTNACNGCNCTNTGGGGYAAG...TTT', IUPACAmbiguousDNA())


<a id='2.2'></a>
### 2.2 Imprimir la información de una secuencia
[Regresar al índice](#indice)

Usar la función <code>dir()</code> en un sólo objeto record para decubrir sus atributos. 

Imprimir el ID, nombre y descripción de la la entrada de globina *sickle cell*

In [25]:
# Usé <dir(gb_record)> para identificaar los atributos del record
for gb_record in SeqIO.parse('outputs/sickle.gb', 'genbank'):
    print('ID  \t\t', gb_record.id)
    print('Nombre\t\t', gb_record.name)
    print('Descripción\t', gb_record.description)

ID  		 M25079.1
Nombre		 HUMBETGLA
Descripción	 Human sickle cell beta-globin mRNA, complete cds


<a id='2.3'></a>
### 2.3 Escribir un archivo FASTA
[Regresar al índice](#indice)

Guardar la entrada de *GenBank* del ejercicio 2.1 en un archivo FASTA usando la función <code>SeqIO.write()</code>

In [26]:
SeqIO.write(gb_record, 'outputs/sickle.fasta', 'fasta')

1

In [27]:
%%bash 
less outputs/sickle.fasta

>M25079.1 Human sickle cell beta-globin mRNA, complete cds
ATGGTNCAYYTNACNCCNGTGGAGAAGTCYGCYGTNACNGCNCTNTGGGGYAAGGTNAAY
GTGGATGAAGYYGGYGGYGAGGCCCTGGGCAGNCTGCTNGTGGTCTACCCTTGGACCCAG
AGGTTCTTNGANTCNTTYGGGGATCTGNNNACNCCNGANGCAGTTATGGGCAACCCTAAG
GTGAAGGCTCATGGCAAGAAAGTGCTCGGTGCCTTTAGTGATGGCCTGGCTCACCTGGAC
AACCTCAAGGGCACCTTTGCCACACTGAGTGAGCTGCACTGTGACAAGCTNCAYGTGGAT
CCTGAGAACTTCAGGCTNCTNGGCAACGTGYTNGTCTGYGTGCTGGCCCATCACTTTGGC
AAAGAATTCACCCCACCAGTGCANGCNGCCTATCAGAAAGTGGTNGCTGGTGTNGCTAAT
GCCCTGGCCCACAAGTATCACTAAGCTNGCYTTYTTGYTGTCCAATTT


<a id='2.4'></a>
### 2.4 Imprimir la infromación de múltiples secuencias
[Regresar al índice](#indice)

Imprimir el ID, nombre y descripción de todas las *beta-globinas* humanas.

In [7]:
count = 0
# No me permitió guardar el parse en una variable por lo que tuve que repetirlo cada vez que lo necesitara
for gb_record in SeqIO.parse('outputs/beta-globins.gb', 'gb'):
    count += 1
    print('\nRecord #{}'.format(count))
    print('ID  \t\t', gb_record.id)
    print('Nombre\t\t', gb_record.name)
    print('Descripción\t', gb_record.description)
    
    
print('\n\nSe obtuvieron un total de {} records de beta-globinas humanas'.format(count))


Record #1
ID  		 MG657341.1
Nombre		 MG657341
Descripción	 Homo sapiens hemoglobin subunit beta (HBB) gene, complete cds

Record #2
ID  		 MG675219.1
Nombre		 MG675219
Descripción	 Homo sapiens hemoglobin subunit beta (HBB) gene, complete cds

Record #3
ID  		 MF462285.1
Nombre		 MF462285
Descripción	 Synthetic construct clone NFAT-CBR red-shifted luciferase (CBR) gene, complete cds

Record #4
ID  		 EU760950.1
Nombre		 EU760950
Descripción	 Homo sapiens isolate TAL45 truncated beta globin gene, complete cds

Record #5
ID  		 EU760937.1
Nombre		 EU760937
Descripción	 Homo sapiens isolate TAL29 truncated beta globin gene, complete cds

Record #6
ID  		 EU760934.1
Nombre		 EU760934
Descripción	 Homo sapiens isolate TAL26 truncated beta globin gene, complete cds

Record #7
ID  		 AH001475.2
Nombre		 AH001475
Descripción	 Homo sapiens beta-globin gene, complete cds

Record #8
ID  		 AH005275.2
Nombre		 AH005275
Descripción	 Homo sapiens chromosome 16 hemoglobin zeta (HBZ) gene, complete c


Record #116
ID  		 AF339409.1
Nombre		 AF339409
Descripción	 Homo sapiens haplotype T4 delta-globin (HBD) gene, complete cds

Record #117
ID  		 AF339408.1
Nombre		 AF339408
Descripción	 Homo sapiens haplotype T3 delta-globin (HBD) gene, complete cds

Record #118
ID  		 AF339407.1
Nombre		 AF339407
Descripción	 Homo sapiens haplotype T2 delta-globin (HBD) gene, complete cds

Record #119
ID  		 AF339406.1
Nombre		 AF339406
Descripción	 Homo sapiens haplotype T1 delta-globin (HBD) gene, complete cds

Record #120
ID  		 AF339405.1
Nombre		 AF339405
Descripción	 Homo sapiens haplotype R5 delta-globin (HBD) gene, complete cds

Record #121
ID  		 AF339404.1
Nombre		 AF339404
Descripción	 Homo sapiens haplotype R4 delta-globin (HBD) gene, complete cds

Record #122
ID  		 AF339403.1
Nombre		 AF339403
Descripción	 Homo sapiens haplotype R3 delta-globin (HBD) gene, complete cds

Record #123
ID  		 AF339402.1
Nombre		 AF339402
Descripción	 Homo sapiens haplotype R2 delta-globin (HBD) gene, compl

<a id='2.5'></a>
### 2.5 Filtrar las entradas de secuencia
[Regresar al índice](#indice)

Imprimir la misma información del ejercicio 2.4 , pero no muestrar las entradas no-globina. Si la descripción contiene *'vector'* o *'isolate'* no imprimir nada.

In [9]:
count = 0
avoidIsolate = 0
avoidVector = 0

for gb_record in SeqIO.parse('outputs/beta-globins.gb', 'gb'):
    desc = gb_record.description
    # Separar la descripción por palabras para detectar 'isolate' o 'vector'
    desc = desc.split()
    if 'isolate' in desc:
        avoidIsolate += 1
    elif 'vector' in desc:
        avoidVector += 1
    else:
        count += 1
        print('\nRecord #{}'.format(count))
        print('ID  \t\t', gb_record.id)
        print('Nombre\t\t', gb_record.name)
        print('Descripción\t', gb_record.description)
        
        
print('\n\nSe ignoraron {} isolates y {} vectores'.format(avoidIsolate, avoidVector))


Record #1
ID  		 MG657341.1
Nombre		 MG657341
Descripción	 Homo sapiens hemoglobin subunit beta (HBB) gene, complete cds

Record #2
ID  		 MG675219.1
Nombre		 MG675219
Descripción	 Homo sapiens hemoglobin subunit beta (HBB) gene, complete cds

Record #3
ID  		 MF462285.1
Nombre		 MF462285
Descripción	 Synthetic construct clone NFAT-CBR red-shifted luciferase (CBR) gene, complete cds

Record #4
ID  		 AH001475.2
Nombre		 AH001475
Descripción	 Homo sapiens beta-globin gene, complete cds

Record #5
ID  		 AH005275.2
Nombre		 AH005275
Descripción	 Homo sapiens chromosome 16 hemoglobin zeta (HBZ) gene, complete cds; HBZP and HBAP1 pseudogenes, complete sequence; and hemoglobin alpha 2 (HBA2) and hemoglobin alpha 1 (HBA1) genes, complete cds

Record #6
ID  		 AH002981.2
Nombre		 AH002981
Descripción	 Homo sapiens S100 protein beta subunit (S100-beta) gene, complete cds

Record #7
ID  		 AH010109.2
Nombre		 AH010109
Descripción	 Homo sapiens odorant receptor HOR3'beta1, odorant receptor HOR3

<a id='3'></a>
## 3. *Bio.Seq* y *Bio.SeqRecord*: Extracción de exones
[Regresar al índice](#indice)

Secuencia proteica de *beta-globina*

<a id='3.1'></a>
### 3.1 Secuencia ADN
[Regresar al índice](#indice)

Leer secuencia cADN de globina *sickle cell*. Usar atributo <code>seq</code> del objeto <code>SeqRecord</code> del ejercicio anterior.

In [11]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

sickleInfo = SeqIO.read('outputs/sickle.fasta', 'fasta')
sickleDNA = sickleInfo.seq

print('Secuencia cADN de globina sickle cell\n\n{}'.format(sickleDNA))

Secuencia cADN de globina sickle cell

ATGGTNCAYYTNACNCCNGTGGAGAAGTCYGCYGTNACNGCNCTNTGGGGYAAGGTNAAYGTGGATGAAGYYGGYGGYGAGGCCCTGGGCAGNCTGCTNGTGGTCTACCCTTGGACCCAGAGGTTCTTNGANTCNTTYGGGGATCTGNNNACNCCNGANGCAGTTATGGGCAACCCTAAGGTGAAGGCTCATGGCAAGAAAGTGCTCGGTGCCTTTAGTGATGGCCTGGCTCACCTGGACAACCTCAAGGGCACCTTTGCCACACTGAGTGAGCTGCACTGTGACAAGCTNCAYGTGGATCCTGAGAACTTCAGGCTNCTNGGCAACGTGYTNGTCTGYGTGCTGGCCCATCACTTTGGCAAAGAATTCACCCCACCAGTGCANGCNGCCTATCAGAAAGTGGTNGCTGGTGTNGCTAATGCCCTGGCCCACAAGTATCACTAAGCTNGCYTTYTTGYTGTCCAATTT


<a id='3.2'></a>
### 3.2 Transcribir ADN a ARN
[Regresar al índice](#indice)

Transcribir la secuencia cADN de globina *sickle cell* a ARN e imprímelo. Usar el método <code>transcribe()</code> del objeto <code>Seq</code>.

In [12]:
sickleRNA = sickleDNA.transcribe()
print('Secuencia ARN de globina sickle cell\n\n{}'.format(sickleRNA))

Secuencia ARN de globina sickle cell

AUGGUNCAYYUNACNCCNGUGGAGAAGUCYGCYGUNACNGCNCUNUGGGGYAAGGUNAAYGUGGAUGAAGYYGGYGGYGAGGCCCUGGGCAGNCUGCUNGUGGUCUACCCUUGGACCCAGAGGUUCUUNGANUCNUUYGGGGAUCUGNNNACNCCNGANGCAGUUAUGGGCAACCCUAAGGUGAAGGCUCAUGGCAAGAAAGUGCUCGGUGCCUUUAGUGAUGGCCUGGCUCACCUGGACAACCUCAAGGGCACCUUUGCCACACUGAGUGAGCUGCACUGUGACAAGCUNCAYGUGGAUCCUGAGAACUUCAGGCUNCUNGGCAACGUGYUNGUCUGYGUGCUGGCCCAUCACUUUGGCAAAGAAUUCACCCCACCAGUGCANGCNGCCUAUCAGAAAGUGGUNGCUGGUGUNGCUAAUGCCCUGGCCCACAAGUAUCACUAAGCUNGCYUUYUUGYUGUCCAAUUU


<a id='3.3'></a>
### 3.3 Traducir ARN a proteína
[Regresar al índice](#indice)

Traduce la secuencia ARN da proteína e imprímela. Usar el método <code>translate()</code> del objeto <code>Seq</code>. Guarda al secuencia en un archivo separado.

In [35]:
sickleProt = sickleRNA.translate()
sickleInfo.seq = sickleProt

SeqIO.write(sickleInfo, 'outputs/sickleProt.fasta', 'fasta')

1

In [36]:
%%bash
less outputs/sickleProt.fasta

>M25079.1 Human sickle cell beta-globin mRNA, complete cds
MVHXTPVEKSAVTALWGKVNVDEXGGEALGXLLVVYPWTQRFXXSFGDLXTPXAVMGNPK
VKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVXVCVLAHHFG
KEFTPPVXAAYQKVVAGVANALAHKYH*AXXXXVQF


<a id='3.4'></a>
### 3.4 Analizar las anotaciones de *beta-globina*
[Regresar al índice](#indice)

En el archivo con múltiples globinas, encontrar la entrada de *beta-globina* con *accession*  <code>L26462</code>. Usar el campo  <code>r.annotations['accessions']</code> en el objeto <code>SeqRecord</code>. Escribir la entrada en un archivo *GenBank* separado.

In [13]:
for gb_record in SeqIO.parse('outputs/beta-globins.gb', 'gb'):
    # Buscar diractamente con el accession number
    if gb_record.annotations['accessions'] == ['L26462']:
        SeqIO.write(gb_record, 'outputs/healthy_betaglobin.gb', 'genbank')

In [14]:
%%bash
head outputs/healthy_betaglobin.gb

LOCUS       HUMBETGLOA              3002 bp    DNA     linear   PRI 26-AUG-1994
DEFINITION  Human haplotype C4 beta-globin gene, complete cds.
ACCESSION   L26462
VERSION     L26462.1
KEYWORDS    beta-globin.
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.


<a id='3.5'></a>
### 3.5 Extraer *features* de la secuencia
[Regresar al índice](#indice)

Imprimir todas las *features* de la entrada <code>L26462</code>. Usar el campo <code>r.features</code> en un objeto <code>SeqRecord</code>.

In [16]:
count = 0

for gb_record in SeqIO.parse('outputs/healthy_betaglobin.gb', 'gb'):
    features = gb_record.features
    for feature in features:
        count += 1
        print(feature)
        
print('Se imprimieron un total de {} features'.format(count))

type: source
location: [0:3002](+)
qualifiers:
    Key: db_xref, Value: ['taxon:9606']
    Key: haplotype, Value: ['C4']
    Key: mol_type, Value: ['genomic DNA']
    Key: note, Value: ['sequence found in a Melanesian population']
    Key: organism, Value: ['Homo sapiens']

type: variation
location: [110:111](+)
qualifiers:
    Key: replace, Value: ['t']

type: variation
location: [262:263](+)
qualifiers:
    Key: note, Value: ['Rsa I polymorphism']
    Key: replace, Value: ['t']

type: variation
location: [272:273](+)
qualifiers:
    Key: replace, Value: ['c']

type: variation
location: [285:287](+)
qualifiers:
    Key: note, Value: ['2 bp insertion of AT']
    Key: replace, Value: ['']

type: variation
location: [287:288](+)
qualifiers:
    Key: replace, Value: ['t']

type: variation
location: [294:296](+)
qualifiers:
    Key: note, Value: ['1 bp deletion of C or 2 bp deletion of CT']
    Key: replace, Value: ['']

type: variation
location: [346:347](+)
qualifiers:
    Key: replace, 

<a id='3.6'></a>
### 3.6 Extraer exones
[Regresar al índice](#indice)

Imprimir todos las *features* de los exones y sus atributos <code>start</code>, <code>end</code> y <code>nofuzzy_start</code>, <code>nofuzzy_end</code>. Por cada exon extraer el intervalo.

Usar los índices para extraer porciones de la secuencia completa, concatena todos las secuencias de exones en una misma cadena.

In [17]:
healthyDNA = ''
count = 0 

for gb_record in SeqIO.parse('outputs/healthy_betaglobin.gb', 'gb'):
    seq = gb_record.seq
    features = gb_record.features
    for feature in features:
        count += 1
        if feature.type == 'exon':
            print('{}start:  {}'.format(feature, feature.location.start))
            print('end:  {}'.format(feature.location.end))
            print('nofuzzy_start:  {}'.format(feature.location.nofuzzy_start))
            print('nofuzzy_end:  {}\n'.format(feature.location.nofuzzy_end))
            healthyDNA += seq[feature.location.nofuzzy_start:feature.location.nofuzzy_end]

print('Se imprimieron un total de {} features'.format(count))

type: exon
location: [<865:957](+)
qualifiers:
    Key: number, Value: ['1']
start:  <865
end:  957
nofuzzy_start:  865
nofuzzy_end:  957

type: exon
location: [1087:1310](+)
qualifiers:
    Key: number, Value: ['2']
start:  1087
end:  1310
nofuzzy_start:  1087
nofuzzy_end:  1310

type: exon
location: [2160:>2289](+)
qualifiers:
    Key: number, Value: ['3']
start:  2160
end:  >2289
nofuzzy_start:  2160
nofuzzy_end:  2289

Se imprimieron un total de 26 features


<a id='3.7'></a>
### 3.7 Extraer la secuencia proteica de *beta-globina*
[Regresar al índice](#indice)

Transcribir y traducir la secuencia de exones concatenados e imprimirla.

In [53]:
healthyRNA = healthyDNA.transcribe()
print('ARN sano de exones concatenados:\n{}'.format(healthyRNA))

healthyProt = healthyRNA.translate()
print('\nProteína sana de exones concatenados:\n{}'.format(healthyProt))

ARN sano de exones concatenados:
AUGGUGCAUCUGACUCCUGAGGAGAAGUCUGCCGUUACUGCCCUGUGGGGCAAGGUGAACGUGGAUGAAGUUGGUGGUGAGGCCCUGGGCAGGCUGCUGGUGGUCUACCCUUGGACCCAGAGGUUCUUUGAGUCCUUUGGGGAUCUGUCCACUCCUGAUGCUGUUAUGGGCAACCCUAAGGUGAAGGCUCAUGGCAAGAAAGUGCUCGGUGCCUUUAGUGAUGGCCUGGCUCACCUGGACAACCUCAAGGGCACCUUUGCCACACUGAGUGAGCUGCACUGUGACAAGCUGCACGUGGAUCCUGAGAACUUCAGGCUCCUGGGCAACGUGCUGGUCUGUGUGCUGGCCCAUCACUUUGGCAAAGAAUUCACCCCACCAGUGCAGGCUGCCUAUCAGAAAGUGGUGGCUGGUGUGGCUAAUGCCCUGGCCCACAAGUAUCACUAA

Proteína sana de exones concatenados:
MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH*


<a id='3.8'></a>
### 3.8 Resultados
[Regresar al índice](#indice)

Imprimir la secuencia de *sickle cell* y sana de *beta-globina* en líneas subsecuentes o en un archivo de texto. Además imprimir las dos secuencias proteicas correspondientes en lineas subsecuentes.


In [55]:
# Función para imprimir las secuencias intercaladas para visualizar las diferencias
def printSubsecuentSeq (healthySeq, sickleSeq):
    if len(healthySeq) > len(sickleSeq):
        stop = len(healthySeq)
    else:
        stop = len(sickleSeq)
    start = 0
    for i in range(60, stop, 60):
        print('\tH-> {}\n\tS-> {}\n'.format(healthySeq[start:i], sickleSeq[start:i]))
        start = i        

In [56]:
print('Paired DNA sequences:\n\tH-> healthy sequence\n\tS-> sickle sequence\n\n')
printSubsecuentSeq(healthyDNA, sickleDNA)

Paired DNA sequences:
	H-> healthy sequence
	S-> sickle sequence


	H-> ATGGTGCATCTGACTCCTGAGGAGAAGTCTGCCGTTACTGCCCTGTGGGGCAAGGTGAAC
	S-> ATGGTNCAYYTNACNCCNGTGGAGAAGTCYGCYGTNACNGCNCTNTGGGGYAAGGTNAAY

	H-> GTGGATGAAGTTGGTGGTGAGGCCCTGGGCAGGCTGCTGGTGGTCTACCCTTGGACCCAG
	S-> GTGGATGAAGYYGGYGGYGAGGCCCTGGGCAGNCTGCTNGTGGTCTACCCTTGGACCCAG

	H-> AGGTTCTTTGAGTCCTTTGGGGATCTGTCCACTCCTGATGCTGTTATGGGCAACCCTAAG
	S-> AGGTTCTTNGANTCNTTYGGGGATCTGNNNACNCCNGANGCAGTTATGGGCAACCCTAAG

	H-> GTGAAGGCTCATGGCAAGAAAGTGCTCGGTGCCTTTAGTGATGGCCTGGCTCACCTGGAC
	S-> GTGAAGGCTCATGGCAAGAAAGTGCTCGGTGCCTTTAGTGATGGCCTGGCTCACCTGGAC

	H-> AACCTCAAGGGCACCTTTGCCACACTGAGTGAGCTGCACTGTGACAAGCTGCACGTGGAT
	S-> AACCTCAAGGGCACCTTTGCCACACTGAGTGAGCTGCACTGTGACAAGCTNCAYGTGGAT

	H-> CCTGAGAACTTCAGGCTCCTGGGCAACGTGCTGGTCTGTGTGCTGGCCCATCACTTTGGC
	S-> CCTGAGAACTTCAGGCTNCTNGGCAACGTGYTNGTCTGYGTGCTGGCCCATCACTTTGGC

	H-> AAAGAATTCACCCCACCAGTGCAGGCTGCCTATCAGAAAGTGGTGGCTGGTGTGGCTAAT
	S-> AAAGAATTCACCCCACCAGTGCANGCNGCCTATCAGAAAGTGGTNGCTGGTGTNGCTAAT



In [57]:
print('Paired protein sequences:\n\tH-> healthy sequence\n\tS-> sickle sequence\n\n')
printSubsecuentSeq(healthyProt, sickleProt)

Paired protein sequences:
	H-> healthy sequence
	S-> sickle sequence


	H-> MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPK
	S-> MVHXTPVEKSAVTALWGKVNVDEXGGEALGXLLVVYPWTQRFXXSFGDLXTPXAVMGNPK

	H-> VKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFG
	S-> VKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVXVCVLAHHFG



<div class="alert alert-block alert-success">
**¿Qué diferencias encuentras?**
<hr>
Las diferencias se deben a cambios en las bases generan una reducción de la proteína por codones stop (X) en la primer mitad de la secuencia.
</div>

<a id='4'></a>
## 4. Coincidencia de patrones: Identificar sitios de restricción
[Regresar al índice](#indice)

Enzimas que discriminan globina *sickle* de globiona *no-sickle*

<a id='4.1'></a>
### 4.1 Familiarización con expresiones regulares
[Regresar al índice](#indice)

Hacer los primeros 3-4 ejercicios en [RegexOne website](http://www.regexone.com/)

**Ejercicio 3**
<img src= "extras/ejercicio3_regexone.png">


**Ejercicio 4**
<img src= "extras/ejercicio4_regexone.png">


<a id='4.2'></a>
### 4.2 Buscar el codón de inicio
[Regresar al índice](#indice)

Usae la función <code>re.search()</code> para localizar el codon de inicio (ATG) en la secuencia cADN de la *beta-globina* sana. 

In [58]:
import re

sickleDNA = str(sickleDNA)
healthyDNA = str(healthyDNA)

match = re.search("ATG", healthyDNA)
if match: 
    print('El codón de inicio {} está en la posición {} del cADN de beta-globulina sana'.format(match.group(), match.span()))

El codón de inicio ATG está en la posición (0, 3) del cADN de beta-globulina sana


<a id='4.3'></a>
### 4.3 Buscar sitio de restricción
[Regresar al índice](#indice)

Crear una expresión regular utilizando la función <code>re.compile()</code> para la enzima de restricción **Ddel** que corta en <code>NN^CTNAG</code>). Buscar con la expresión regular en ambas secuencias (sana y *sickle cell*) de la *beta-globina*.

Si el método de búsqueda regresa un *match*, imprimir el inicio y  final de ambas secuencias.

In [59]:
# Función para evaluar la presencia de ambos patrones simultáneamente
def findRestrictionSite(regex, healthySeq, sickleSeq):
    pattern = re.compile(regex, flags = re.M)
    matchHealthy = pattern.search(healthySeq)
    matchSickle = pattern.search(sickleSeq)
    if matchHealthy:
        print('\tSana -> Reconoce {} y corta en la posición {}.'.format(matchHealthy.group(), matchHealthy.span()))
    else:
        print('\tSana -> No tiene sitio de restricción.')
   
    if matchSickle:
        print('\tSickle -> Reconoce {} y corta en la posición {}.'.format(matchSickle.group(), matchSickle.span()))
    else:
        print('\tSickle -> No tiene sitio de restricción.')
   

In [60]:
print('La enzima Ddel en célula:\n')
findRestrictionSite('..CT.AG', healthyDNA, sickleDNA)

La enzima Ddel en célula:

	Sana -> Reconoce TCCTGAG y corta en la posición (14, 21).
	Sickle -> Reconoce CCCTAAG y corta en la posición (173, 180).


<a id='4'></a>
### 4.4 Utilizar más sitios de restricción
[Regresar al índice](#indice)

Usar los patrones:

- HinfI (GTNNAC)
- BceAI (ACGGCNNNNNNNNNNNNN)
- BseRI (GAGGAGNNNNNNNNNN)
- EcoRI (GAATTC)
- MstII (CCTNAGG) 

en ambas secuencias de ADN. 

In [61]:
print('La enzima HinfI en célula:\n')
findRestrictionSite('GT..AC', healthyDNA, sickleDNA)

La enzima HinfI en célula:

	Sana -> Reconoce GTGAAC y corta en la posición (54, 60).
	Sickle -> Reconoce GTCTAC y corta en la posición (102, 108).


In [62]:
print('La enzima BceAI en célula:\n')
findRestrictionSite('ACGGC.{13}', healthyDNA, sickleDNA)

La enzima BceAI en célula:

	Sana -> No tiene sitio de restricción.
	Sickle -> No tiene sitio de restricción.


In [63]:
print('La enzima BseRI en célula:\n')
findRestrictionSite('.GAGGAG.{10}', healthyDNA, sickleDNA)

La enzima BseRI en célula:

	Sana -> Reconoce TGAGGAGAAGTCTGCCG y corta en la posición (17, 34).
	Sickle -> No tiene sitio de restricción.


In [64]:
print('La enzima EcoRI en célula:\n')
findRestrictionSite('GAATTC', healthyDNA, sickleDNA)

La enzima EcoRI en célula:

	Sana -> Reconoce GAATTC y corta en la posición (363, 369).
	Sickle -> Reconoce GAATTC y corta en la posición (363, 369).


In [65]:
print('La enzima MstII en célula:\n')
findRestrictionSite('CCT.AGG', healthyDNA, sickleDNA)

La enzima MstII en célula:

	Sana -> Reconoce CCTGAGG y corta en la posición (15, 22).
	Sickle -> Reconoce CCTAAGG y corta en la posición (174, 181).


<div class="alert alert-block alert-success">
**¿Qué enzima de restricción utilizarías específicamente para identificar portadores del gen de anemia falciforme?**
<hr>
La enzima BseRI porque no tiene sitios de restricción para la secuancia de anemia falciforme, por lo que no se vería afectada y podríamos identificarla ya que se mantiene completa a diferencia de la beta-globina sana. 
</div>

<a id='4.5'></a>
### 4.5 Reemplazar los caracteres faltantes
[Regresar al índice](#indice)

Para facilitar el análisis de restricción, reemplazar las N's en el ADN *sickle cell* por las posiciones correspondientes en el ADN sano. Imprimir la secuencia resultante.

In [36]:
flag = True
correctedSickleDNA = sickleDNA

while flag:
    try:
        indexN  = correctedSickleDNA.index('N')
        correctedSickleDNA = correctedSickleDNA[:indexN-1] + healthyDNA[indexN] + correctedSickleDNA[indexN + 1:]
    except ValueError:
        flag = False
        
print("Secuencia de ADN de sickle cell sin N's\n\n",correctedSickleDNA)

Secuencia de ADN de sickle cell sin N's

 ATGGGCAYYTAACTGTGGAGAAGTCYGCYGCAGGTCCTGGGGYAAGGGAAYGTGGATGAAGYYGGYGGYGAGGCCCTGGGCACCTGCGGTGGTCTACCCTTGGACCCAGAGGTTCTCGGTGTTYGGGGATCTCATCTGGGCAGTTATGGGCAACCCTAAGGTGAAGGCTCATGGCAAGAAAGTGCTCGGTGCCTTTAGTGATGGCCTGGCTCACCTGGACAACCTCAAGGGCACCTTTGCCACACTGAGTGAGCTGCACTGTGACAAGCGCAYGTGGATCCTGAGAACTTCAGGCGCAGGCAACGTGYTGTCTGYGTGCTGGCCCATCACTTTGGCAAAGAATTCACCCCACCAGTGCCGAGCCTATCAGAAAGTGGGGCTGGTGTGCTAATGCCCTGGCCCACAAGTATCACTAAGCTGCYTTYTTGYTGTCCAATTT


<a id='5'></a>
## Resultados: Archivos *output* generados

Nombre de los archivos generados a lo largo del ejercicio

In [72]:
%%bash 
# Lista de 5 outputs generados
ls -h outputs/

beta-globins.gb
healthy_betaglobin.gb
sickle.fasta
sickle.gb
sickleProt.fasta


In [33]:
%%bash 
# Archivo genbank con cada record de la búsqueda y extracción para beta-globinas humanas
head outputs/beta-globins.gb

LOCUS       MG657341                1606 bp    DNA     linear   PRI 22-OCT-2018
DEFINITION  Homo sapiens hemoglobin subunit beta (HBB) gene, complete cds.
ACCESSION   MG657341
VERSION     MG657341.1
KEYWORDS    .
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.


In [34]:
%%bash  
# Archivo genbank de beta-globina humana sana
head outputs/healthy_betaglobin.gb

LOCUS       HUMBETGLOA              3002 bp    DNA     linear   PRI 26-AUG-1994
DEFINITION  Human haplotype C4 beta-globin gene, complete cds.
ACCESSION   L26462
VERSION     L26462.1
KEYWORDS    beta-globin.
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.


In [39]:
%%bash  
# Archivo con la secuencia cADN de beta-globina sickle cell (sin sustitución de Ns)
head outputs/sickle.fasta

>M25079.1 Human sickle cell beta-globin mRNA, complete cds
ATGGTNCAYYTNACNCCNGTGGAGAAGTCYGCYGTNACNGCNCTNTGGGGYAAGGTNAAY
GTGGATGAAGYYGGYGGYGAGGCCCTGGGCAGNCTGCTNGTGGTCTACCCTTGGACCCAG
AGGTTCTTNGANTCNTTYGGGGATCTGNNNACNCCNGANGCAGTTATGGGCAACCCTAAG
GTGAAGGCTCATGGCAAGAAAGTGCTCGGTGCCTTTAGTGATGGCCTGGCTCACCTGGAC
AACCTCAAGGGCACCTTTGCCACACTGAGTGAGCTGCACTGTGACAAGCTNCAYGTGGAT
CCTGAGAACTTCAGGCTNCTNGGCAACGTGYTNGTCTGYGTGCTGGCCCATCACTTTGGC
AAAGAATTCACCCCACCAGTGCANGCNGCCTATCAGAAAGTGGTNGCTGGTGTNGCTAAT
GCCCTGGCCCACAAGTATCACTAAGCTNGCYTTYTTGYTGTCCAATTT


In [38]:
%%bash  
# Archivo genbank de mARN de beta-globina sickle cell
head outputs/sickle.gb

LOCUS       HUMBETGLA                468 bp    mRNA    linear   PRI 27-APR-1993
DEFINITION  Human sickle cell beta-globin mRNA, complete cds.
ACCESSION   M25079
VERSION     M25079.1
KEYWORDS    .
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.


In [37]:
%%bash  
# Archivo fasta con secuencia proteica de beta-globina sickle cell
head outputs/sickleProt.fasta

>M25079.1 Human sickle cell beta-globin mRNA, complete cds
MVHXTPVEKSAVTALWGKVNVDEXGGEALGXLLVVYPWTQRFXXSFGDLXTPXAVMGNPK
VKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVXVCVLAHHFG
KEFTPPVXAAYQKVVAGVANALAHKYH*AXXXXVQF


<a id='6'></a>
## Conclusiones

La combinación de herramientas bioinformáticas para el manejo de información y objetos como secuencias, nos permite desarrollar aplicaciones nuevas. Este ejercicio a pesar de parecer simple a grandes rasgos, en algún momento representó un parteaguas para el diagnóstico de anemia falciforme o alguna otra enfermedad. 

Existen pruebas que requieren de desarrollo e interpretación bioinformática, seguramente en laboratorio se pueden descubrir interacciones de enzimas de restricción a partir de distintos experimentos. Sin emabargo, la bioinformática facilita la la identificación de estos sitios basándose en la manipulación de objetos y combinándolo con conocimientos. La combionación de estrategias experimentales y herramientas bioinformáticas permitirán el desarrollo de una ciencia más eficiente.